In [1]:
import pandas as pd
import os
from recsysUtility import RecSysUtility

# Come primo step devo generare un dataset adatto alla MF

In [2]:
label = 'Retweet_with_comment'
training_file = '//fmnas/Dataset/new/training.tsv'
output_file = '//fmnas/Dataset/mf_{}.csv'.format(label)
rsUtils = RecSysUtility(training_file)

if(not os.path.exists(output_file)):
    os.system('touch {}'.format(output_file))
else:
    os.system('rm {}'.format(output_file))
    os.system('touch {}'.format(output_file))
c_size = 1000000
i = 0
for df_chunk in pd.read_csv(training_file, sep='\u0001', header=None, chunksize=c_size):
    print('Ho analizzato {} righe'.format(c_size*i))
    df_chunk = rsUtils.process_chunk_tsv(df_chunk)
    col_label = label + '_engagement_timestamp'
    df_chunk = df_chunk[df_chunk[col_label] > 0]
    df_chunk = df_chunk[['User_id', 'User_id_engaging', 'Hashtags']]
    df_chunk.to_csv(output_file, mode='a', header=False, index=False)
    i += 1

Ho analizzato 0 righe
Ho analizzato 1000000 righe
Ho analizzato 2000000 righe
Ho analizzato 3000000 righe
Ho analizzato 4000000 righe
Ho analizzato 5000000 righe
Ho analizzato 6000000 righe
Ho analizzato 7000000 righe
Ho analizzato 8000000 righe
Ho analizzato 9000000 righe
Ho analizzato 10000000 righe
Ho analizzato 11000000 righe
Ho analizzato 12000000 righe
Ho analizzato 13000000 righe
Ho analizzato 14000000 righe
Ho analizzato 15000000 righe
Ho analizzato 16000000 righe
Ho analizzato 17000000 righe
Ho analizzato 18000000 righe
Ho analizzato 19000000 righe
Ho analizzato 20000000 righe
Ho analizzato 21000000 righe
Ho analizzato 22000000 righe
Ho analizzato 23000000 righe
Ho analizzato 24000000 righe
Ho analizzato 25000000 righe
Ho analizzato 26000000 righe
Ho analizzato 27000000 righe
Ho analizzato 28000000 righe
Ho analizzato 29000000 righe
Ho analizzato 30000000 righe
Ho analizzato 31000000 righe
Ho analizzato 32000000 righe
Ho analizzato 33000000 righe
Ho analizzato 34000000 righe
H

In [7]:
def generate_mf_csv(self, label, output_path='//fmnas/Dataset/'):
    output_file =  output_file + 'mf_{}.csv'.format(label)
    rsUtils = RecSysUtility(training_file)

    if(not os.path.exists(output_file)):
        os.system('touch {}'.format(output_file))
    else:
        os.system('rm {}'.format(output_file))
        os.system('touch {}'.format(output_file))
    c_size = 1000000
    i = 0
    for df_chunk in pd.read_csv(self.training_file, sep='\u0001', header=None, chunksize=c_size):
        print('Ho analizzato {} righe'.format(c_size*i))
        df_chunk = rsUtils.process_chunk_tsv(df_chunk)
        col_label = label + '_engagement_timestamp'
        df_chunk = df_chunk[df_chunk[col_label] > 0]
        df_chunk = df_chunk[['User_id', 'User_id_engaging', 'Hashtags']]
        df_chunk.to_csv(output_file, mode='a', header=False, index=False)
        i += 1
    return

In [11]:
def generate_mf_val(validation_file='//fmnas/Dataset/val.tsv', output_file='//fmnas/Dataset/val_mf.csv'):
    rsUtils = RecSysUtility(training_file)
    if(not os.path.exists(output_file)):
        os.system('touch {}'.format(output_file))
    else:
        os.system('rm {}'.format(output_file))
        os.system('touch {}'.format(output_file))
    c_size = 1000000
    i = 0
    for df_chunk in pd.read_csv(validation_file, sep='\u0001', header=None, chunksize=c_size):
        print('Ho analizzato {} righe'.format(c_size*i))
        df_chunk = rsUtils.process_chunk_tsv(df_chunk, isVal=True)
        df_chunk = df_chunk[['User_id', 'User_id_engaging', 'Hashtags']]
        df_chunk.to_csv(output_file, mode='a', header=False, index=False)
        i += 1
    return

In [12]:
generate_mf_val()

Ho analizzato 0 righe
Ho analizzato 1000000 righe
Ho analizzato 2000000 righe
Ho analizzato 3000000 righe
Ho analizzato 4000000 righe
Ho analizzato 5000000 righe
Ho analizzato 6000000 righe
Ho analizzato 7000000 righe
Ho analizzato 8000000 righe
Ho analizzato 9000000 righe
Ho analizzato 10000000 righe
Ho analizzato 11000000 righe
Ho analizzato 12000000 righe
Ho analizzato 13000000 righe


## Preparo il dataset per LightFM

In [121]:
def create_mf_dataset(df_train, df_test):
    
    #Mi prendo la lista di tutti gli hashtag
    df_full = pd.concat([df_train, df_test], ignore_index=True)
    h_list = []
    for a in df_full['Hashtags'].values:
        if len(a) > 0:
           h_list +=a 
    h_list = list(set(h_list))
    print('Hashtag totali: {}'.format(len(h_list)))
    dataset = Dataset()
    
    # Creo il mapping tra User e Author e le Features
    
    dataset.fit(df_full['User'], df_full['Author'], item_features=h_list)
    num_users, num_items = dataset.interactions_shape()
    print('Num users: {}, num_authors {}.'.format(num_users, num_items))
    #item_features = dataset.build_item_features(list(zip(df_train.Author, df_train.Hashtags)))
        
    # Creo la sparse matrix
    #(interactions_training, weights) = dataset.build_interactions(list(zip(df_train.User, df_train.Author)))                                                         
    
    return dataset

In [122]:
from sklearn.model_selection import train_test_split
from lightfm.data import Dataset
import gc

label = 'Retweet_with_comment'
training_file = 'mf_{}.csv'.format(label)
df_train = pd.read_csv(training_file, header=None)
df_train.columns = ['Author', 'User', 'Hashtags']
df_train.loc[:, 'Hashtags'] = df_train['Hashtags'].apply(lambda x: x.split('|') if x != '0' else [])

In [123]:
df_val = pd.read_csv('val_mf.csv', nrows=1000)
df_val.columns =  ['Author', 'User', 'Hashtags']
df_val.loc[:, 'Hashtags'] = df_val['Hashtags'].apply(lambda x: x.split('|') if x != '0' else [])

In [124]:
dataset_builder = create_mf_dataset(df_train, df_val)

Hashtag totali: 109418
Num users: 866835, num_authors 560883.


Mapping per gli autori (0-1)

In [35]:
len(dataset_builder.mapping()[0])

866118

Mapping per gli items (2-3)

In [37]:
len(dataset_builder.mapping()[2])

609714

### Creo il dataset per il training

In [125]:
(interactions_training, weights) = dataset_builder.build_interactions(list(zip(df_train['User'], df_train['Author'])))  

In [106]:
item_features_train = dataset_builder.build_item_features(list(zip(df_train.Author, df_train.Hashtags)))
item_features_train

<560883x670301 sparse matrix of type '<class 'numpy.float32'>'
	with 794586 stored elements in Compressed Sparse Row format>

In [107]:
from lightfm import LightFM

model = LightFM()
model.fit(interactions_training, item_features=item_features)

### Faccio le prediction

In [108]:
user_mapping = dataset_builder.mapping()[0]
author_mapping = dataset_builder.mapping()[2]

In [109]:
df_val.head()

,Author,User,Hashtags
0,825B003667DF15692B1704F44DAFD1A2,000013315386492275CCBF7AEF293EF0,[]
1,2DEC09796D0EB473244CB7A4FB13A0A0,00001569CB28972FC8173122D9DA162F,[]
2,487BCB8F1677A6D9EEE612940410CBB6,00001607209C5774DF9207A2AC0EED5F,[]
3,C85C39DC8D279E51DF1BBF0391E5EB2E,0000177705514C315F2FC6DFA3872712,[]
4,D0A3A1BDAF414BE91D6D482A021614CC,00001BC70532632181F17B2A65EFD2BA,[]


In [52]:
df_val.columns

Index(['Author', 'User ', 'Hashtags'], dtype='object')

In [110]:
user_test = df_val['User'].to_list()
author_test = df_val['Author'].to_list()
user_test = list(map(user_mapping.get, user_test))
author_test = list(map(author_mapping.get, author_test))

In [115]:
df_val['Hashtags'].to_list()

[[],
 [],
 [],
 [],
 [],
 [],
 ['AC6D3435D9553482AD3DE25576B85B57',
  '699C2389AC28565A67CCD90AD769EB30',
  'F564E2BDCB9A0ABADDD4A9EE053DFCD2'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['F0F2FBE57F08E7C4326682B5EDA63E3E', 'E78674D323461112D0DCF8010AF5AED3'],
 [],
 ['CA9E12A4233C5D315168FCFCDB6CCF00',
  '50E71F214FFC1AB9D5774B35771C4699',
  'DBCF3F98EE8FE34CD201ED7FF74896B6'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['F08B34EB716D7EBABBA8F468B8677427',
  '3163A5EBCEC26EC718403CB280FDEC9E',
  '98E996AA0A2A7D25577CB35B510A1908'],
 ['6AC5277FBFDFD8B273C06C758E3EC39A'],
 ['99CCA670D6FD350E8CF02DA5081D77BE'],
 [],
 [],
 [],
 ['56948147C599FAD7C28C2449C9AE3ABC'],
 [],
 [],
 [],
 [],
 [],
 ['8F948A69BB11C4B2AC7A9D7DCF88F354'],
 [],
 [],
 [],
 [],
 [],
 ['FB5A802DF7D5F25412425F0B91B1D7A7', '02FFE3109EF6E882F264251BF9D63F01'],
 [],
 [],
 ['4A8B492378EC22A125AABBCB35B09DDF'],
 [],
 [],
 [],
 [],
 [],
 ['80D4DF964369F45609A3E4E81FB597EC'],
 [],
 [],
 

In [126]:
item_features_test = dataset_builder.build_item_features(list(zip(df_val['Author'].to_list(), df_val['Hashtags'].to_list())))
item_features_test

<560883x670301 sparse matrix of type '<class 'numpy.float32'>'
	with 561313 stored elements in Compressed Sparse Row format>

In [119]:
import numpy as np
model.predict(user_test,
              author_test,
              item_features=item_features_test)

AssertionError: 

### Valuto il modello appena allenato

In [ ]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

#train_precision = precision_at_k(model, interactions_training, k=10, num_threads=8).mean()
test_precision = precision_at_k(model, interactions_test, k=10, num_threads=8, item_features=item_features_training).mean()

#train_auc = auc_score(model, interactions_training, num_threads=8).mean()
test_auc = auc_score(model, interactions_test, num_threads=8, item_features=item_features_training).mean()

print('Precision: test %.2f.' % (test_precision))
print('AUC: test %.2f.' % (test_auc))

## Aggiungo anche le item features (quindi gli hashtag usati)

In [53]:
item_features = dataset.build_item_features(((x['ISBN'], [x['Book-Author']])
                                              for x in get_book_features()))
item_features = dataset.build_item_features()

Num users: 2808615, num_items 1721003.
